# Персптрон для задачи регрессии

In [21]:
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, random_split
from tqdm import tqdm


In [22]:
data = fetch_california_housing()
X, y = data.data, data.target

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(X_tensor, y_tensor)

total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - (train_size + val_size)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [26]:
class PerceptronRegression(nn.Module):
  def __init__(self, input_size):
    super(PerceptronRegression, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.bn1 = nn.BatchNorm1d(128)

    self.fc2 = nn.Linear(128, 64)
    self.bn2 = nn.BatchNorm1d(64)

    self.fc3 = nn.Linear(64, 32)
    self.bn3 = nn.BatchNorm1d(32)

    self.fc4 = nn.Linear(32, 1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout()

  def forward(self, x):
    x = self.relu(self.bn1(self.fc1(x)))
    x = self.relu(self.bn2(self.fc2(x)))
    x = self.relu(self.bn3(self.fc3(x)))
    x = self.fc4(x)
    return x

In [27]:
input_size = X.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PerceptronRegression(input_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

patience = 10
best_val_loss = float("inf")
patience_counter = 0
model_path = "best_perceptron_regression.pth"

epochs = 100

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for data, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  train_loss = running_loss / len(train_loader)

  model.eval()
  val_loss = 0.0
  with torch.no_grad():
    for data, target in val_loader:
      data, target = data.to(device), target.to(device)
      outputs = model(data)
      loss = criterion(outputs, target)
      val_loss += loss.item()

  val_loss /= len(val_loader)

  print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    patience_counter = 0
    torch.save(model.state_dict(), model_path)
    print(f"✔ Model improved. Saved to {model_path}")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print("Ранняя остановка сработала!")
      break

model.load_state_dict(torch.load(model_path))
model.to(device)
print(f"✅ Best model loaded from {model_path}")

model.eval()
test_loss = 0.0
with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    outputs = model(data)
    loss = criterion(outputs, target)
    test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

Epoch 1/100: 100%|██████████| 387/387 [00:01<00:00, 267.10it/s]


Epoch 1, Train Loss: 0.3700, Val Loss: 0.3066
✔ Model improved. Saved to best_perceptron_regression.pth


Epoch 2/100: 100%|██████████| 387/387 [00:01<00:00, 299.48it/s]


Epoch 2, Train Loss: 0.3084, Val Loss: 0.5735


Epoch 3/100: 100%|██████████| 387/387 [00:01<00:00, 256.46it/s]


...

Epoch 27, Train Loss: 0.2309, Val Loss: 0.2392


Epoch 28/100: 100%|██████████| 387/387 [00:01<00:00, 220.04it/s]


Epoch 28, Train Loss: 0.2365, Val Loss: 0.6728


Epoch 29/100: 100%|██████████| 387/387 [00:01<00:00, 203.18it/s]


Epoch 29, Train Loss: 0.2346, Val Loss: 0.5567
Ранняя остановка сработала!
✅ Best model loaded from best_perceptron_regression.pth
Test Loss: 0.2350
